# Learning the Rules

Suppose we are given a very simple dataset $V$ consisting of vectors in the first quadrant of the plane excluding the origin. That is, our data is a subset of $[0,1]\times[0,1]$. Now, we assign 3 classes to the vectors based on the following rule:
$$
    \begin{bmatrix} * \\ 0 \end{bmatrix} \mapsto \text{ class } 0, \quad \begin{bmatrix} 0 \\ * \end{bmatrix} \mapsto \text{ class } 1, \quad \begin{bmatrix} * \\ * \end{bmatrix} \mapsto \text{ class } 2.
$$

We then ask if a neural network with 2 layers can find a mapping that will identify the classes based on training data.

Note that we can easily construct a method for classifying vectors in $V$ using matrix multiplication. In particular, the three matrices
$$
    A = \begin{bmatrix} 1 & 0 \\ 0 & 0 \\ 0 & 0 \end{bmatrix}, \quad B = \begin{bmatrix} 0 & 0 \\ 0 & 1 \\ 0 & 0 \end{bmatrix}, \quad C = \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1 & 1 \end{bmatrix}
$$
represent mappings to class 0, 1, and 2, respectively.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
N = 10000

In [3]:
col1, col2 = np.zeros(N), np.zeros(N)
col1[:5000] = np.random.rand(5000)
col2[5000:] = np.random.rand(5000)

classes = [0]*5000 + [1]*5000

In [4]:
V = pd.DataFrame({'x1': col1, 'x2' : col2, 'class': classes})

In [5]:
train, test = train_test_split(V, test_size = 0.4)

X_train = train[['x1','x2']].to_numpy()
Y_train = train['class'].to_numpy()

X_test = test[['x1','x2']].to_numpy()
Y_test = test['class'].to_numpy()

In [6]:
layer1 = Dense(4, input_dim = 2, activation = 'relu')
layer2 = Dense(32, activation = 'relu')
output_layer = Dense(1, activation = 'sigmoid')

model = Sequential([layer1, output_layer])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
results = []
times = [25,50,100,200]

for epoch in times:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(X_train, Y_train,
              epochs = epoch,
              batch_size = 250,
              verbose = False);
    
    Y_pred = model.predict_classes(X_test)

    results.append(sum([np.abs(Y_pred[i][0] - Y_test[i]) for i in range(len(Y_test))]))

In [11]:
for i in range(len(times)):
    print('Using {} training epochs'.format(times[i]))
    print('Validation accuracy: {:.4f}%'.format(100*(1-results[i]/len(Y_test))))
    print('Model got {} wrong out of {}\n'.format(results[i], len(Y_test)))

Using 25 training epochs
Validation accuracy: 49.0750%
Model got 2037 wrong out of 4000

Using 50 training epochs
Validation accuracy: 49.0750%
Model got 2037 wrong out of 4000

Using 100 training epochs
Validation accuracy: 49.0750%
Model got 2037 wrong out of 4000

Using 200 training epochs
Validation accuracy: 49.0750%
Model got 2037 wrong out of 4000



In [12]:
print(results)

[2037, 2037, 2037, 2037]


In [9]:
y = model.predict_classes(X_train)

sum([np.abs(y[i][0] - Y_train[i]) for i in range(len(Y_train))])

2963

# Fitting random labels

In this experiment, we'll use the MNIST dataset with randomly assigned class labels. The network is a 2-layer MLP with 1024 units per layer. Using the same randomly assigned labels, we'll examine the difference in test error between the network's performance with and without $L_2$-regularization (weight decay).

In [2]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD

batch_size = 100
num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print(type(y_train))

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples
<class 'numpy.ndarray'>


In [2]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [4]:
epochs = 1000

In [5]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 5s 78us/step - loss: 1.0622 - accuracy: 0.7413 - val_loss: 0.4922 - val_accuracy: 0.8750
Epoch 2/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.4272 - accuracy: 0.8864 - val_loss: 0.3546 - val_accuracy: 0.9037
Epoch 3/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.3455 - accuracy: 0.9037 - val_loss: 0.3081 - val_accuracy: 0.9138
Epoch 4/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.3075 - accuracy: 0.9128 - val_loss: 0.2822 - val_accuracy: 0.9206
Epoch 5/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.2827 - accuracy: 0.9198 - val_loss: 0.2621 - val_accuracy: 0.9262
Epoch 6/1000
60000/60000 [==============================] - 3s 51us/step - loss: 0.2630 - accuracy: 0.9262 - val_loss: 0.2458 - val_accuracy: 0.9308
Epoch 7/1000
60000/60000 [==============================

60000/60000 [==============================] - 3s 48us/step - loss: 0.0191 - accuracy: 0.9967 - val_loss: 0.0650 - val_accuracy: 0.9794
Epoch 110/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.0188 - accuracy: 0.9969 - val_loss: 0.0642 - val_accuracy: 0.9804
Epoch 111/1000
60000/60000 [==============================] - 3s 49us/step - loss: 0.0185 - accuracy: 0.9970 - val_loss: 0.0659 - val_accuracy: 0.9791
Epoch 112/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.0181 - accuracy: 0.9973 - val_loss: 0.0647 - val_accuracy: 0.9792
Epoch 113/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.0178 - accuracy: 0.9971 - val_loss: 0.0646 - val_accuracy: 0.9793
Epoch 114/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.0175 - accuracy: 0.9974 - val_loss: 0.0643 - val_accuracy: 0.9804
Epoch 115/1000
60000/60000 [==============================] - 3s 49us/step - loss: 0.0173 - accuracy: 0.9973 

60000/60000 [==============================] - 3s 47us/step - loss: 0.0044 - accuracy: 0.9999 - val_loss: 0.0680 - val_accuracy: 0.9804
Epoch 218/1000
60000/60000 [==============================] - 3s 47us/step - loss: 0.0043 - accuracy: 0.9999 - val_loss: 0.0682 - val_accuracy: 0.9805
Epoch 219/1000
60000/60000 [==============================] - 3s 47us/step - loss: 0.0043 - accuracy: 0.9999 - val_loss: 0.0683 - val_accuracy: 0.9804
Epoch 220/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.0042 - accuracy: 0.9999 - val_loss: 0.0684 - val_accuracy: 0.9802
Epoch 221/1000
60000/60000 [==============================] - 3s 48us/step - loss: 0.0042 - accuracy: 0.9999 - val_loss: 0.0690 - val_accuracy: 0.9804
Epoch 222/1000
60000/60000 [==============================] - 3s 49us/step - loss: 0.0042 - accuracy: 0.9999 - val_loss: 0.0684 - val_accuracy: 0.9805
Epoch 223/1000
60000/60000 [==============================] - 3s 49us/step - loss: 0.0041 - accuracy: 0.9999 

60000/60000 [==============================] - 2s 37us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0730 - val_accuracy: 0.9814
Epoch 326/1000
60000/60000 [==============================] - 2s 36us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0731 - val_accuracy: 0.9813
Epoch 327/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0733 - val_accuracy: 0.9812
Epoch 328/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0733 - val_accuracy: 0.9815
Epoch 329/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0734 - val_accuracy: 0.9810
Epoch 330/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0733 - val_accuracy: 0.9812
Epoch 331/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0018 - accuracy: 1.0000 

60000/60000 [==============================] - 2s 38us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0750 - val_accuracy: 0.9814
Epoch 380/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0748 - val_accuracy: 0.9814
Epoch 381/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0750 - val_accuracy: 0.9813
Epoch 382/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0751 - val_accuracy: 0.9814
Epoch 383/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0750 - val_accuracy: 0.9814
Epoch 384/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0751 - val_accuracy: 0.9815
Epoch 385/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0014 - accuracy: 1.0000 

...

Epoch 699/1000
60000/60000 [==============================] - 2s 37us/step - loss: 5.1816e-04 - accuracy: 1.0000 - val_loss: 0.0828 - val_accuracy: 0.9816
Epoch 700/1000
60000/60000 [==============================] - 2s 37us/step - loss: 5.1785e-04 - accuracy: 1.0000 - val_loss: 0.0828 - val_accuracy: 0.9817
Epoch 701/1000
60000/60000 [==============================] - 2s 37us/step - loss: 5.1618e-04 - accuracy: 1.0000 - val_loss: 0.0829 - val_accuracy: 0.9816
Epoch 702/1000
60000/60000 [==============================] - 2s 37us/step - loss: 5.1495e-04 - accuracy: 1.0000 - val_loss: 0.0830 - val_accuracy: 0.9816
Epoch 703/1000
60000/60000 [==============================] - 2s 37us/step - loss: 5.1404e-04 - accuracy: 1.0000 - val_loss: 0.0829 - val_accuracy: 0.9816
Epoch 704/1000
60000/60000 [==============================] - 2s 37us/step - loss: 5.1286e-04 - accuracy: 1.0000 - val_loss: 0.0829 - val_accuracy: 0.9814
Epoch 705/1000
60000/60000 [==============================] - 2s 37us/

Epoch 752/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.6295e-04 - accuracy: 1.0000 - val_loss: 0.0837 - val_accuracy: 0.9818
Epoch 753/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.6230e-04 - accuracy: 1.0000 - val_loss: 0.0836 - val_accuracy: 0.9819
Epoch 754/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.6171e-04 - accuracy: 1.0000 - val_loss: 0.0837 - val_accuracy: 0.9818
Epoch 755/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.6022e-04 - accuracy: 1.0000 - val_loss: 0.0836 - val_accuracy: 0.9819
Epoch 756/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.5946e-04 - accuracy: 1.0000 - val_loss: 0.0838 - val_accuracy: 0.9816
Epoch 757/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.5864e-04 - accuracy: 1.0000 - val_loss: 0.0838 - val_accuracy: 0.9815
Epoch 758/1000
60000/60000 [==============================] - 2s 37us/

Epoch 805/1000
60000/60000 [==============================] - 2s 38us/step - loss: 4.1754e-04 - accuracy: 1.0000 - val_loss: 0.0845 - val_accuracy: 0.9819
Epoch 806/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.1614e-04 - accuracy: 1.0000 - val_loss: 0.0846 - val_accuracy: 0.9816
Epoch 807/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.1612e-04 - accuracy: 1.0000 - val_loss: 0.0845 - val_accuracy: 0.9818
Epoch 808/1000
60000/60000 [==============================] - 2s 37us/step - loss: 4.1500e-04 - accuracy: 1.0000 - val_loss: 0.0845 - val_accuracy: 0.9818
Epoch 809/1000
60000/60000 [==============================] - 2s 39us/step - loss: 4.1444e-04 - accuracy: 1.0000 - val_loss: 0.0845 - val_accuracy: 0.9818
Epoch 810/1000
60000/60000 [==============================] - 2s 38us/step - loss: 4.1356e-04 - accuracy: 1.0000 - val_loss: 0.0847 - val_accuracy: 0.9815
Epoch 811/1000
60000/60000 [==============================] - 2s 37us/

Epoch 858/1000
60000/60000 [==============================] - 2s 40us/step - loss: 3.7920e-04 - accuracy: 1.0000 - val_loss: 0.0854 - val_accuracy: 0.9817
Epoch 859/1000
60000/60000 [==============================] - 2s 39us/step - loss: 3.7878e-04 - accuracy: 1.0000 - val_loss: 0.0854 - val_accuracy: 0.9818
Epoch 860/1000
60000/60000 [==============================] - 2s 39us/step - loss: 3.7784e-04 - accuracy: 1.0000 - val_loss: 0.0853 - val_accuracy: 0.9818
Epoch 861/1000
60000/60000 [==============================] - 2s 37us/step - loss: 3.7736e-04 - accuracy: 1.0000 - val_loss: 0.0853 - val_accuracy: 0.9817
Epoch 862/1000
60000/60000 [==============================] - 2s 38us/step - loss: 3.7658e-04 - accuracy: 1.0000 - val_loss: 0.0853 - val_accuracy: 0.9819
Epoch 863/1000
60000/60000 [==============================] - 2s 37us/step - loss: 3.7612e-04 - accuracy: 1.0000 - val_loss: 0.0855 - val_accuracy: 0.9815
Epoch 864/1000
60000/60000 [==============================] - 2s 37us/

Epoch 963/1000
60000/60000 [==============================] - 2s 37us/step - loss: 3.1942e-04 - accuracy: 1.0000 - val_loss: 0.0869 - val_accuracy: 0.9815
Epoch 964/1000
60000/60000 [==============================] - 2s 37us/step - loss: 3.1888e-04 - accuracy: 1.0000 - val_loss: 0.0868 - val_accuracy: 0.9816
Epoch 965/1000
60000/60000 [==============================] - 2s 38us/step - loss: 3.1863e-04 - accuracy: 1.0000 - val_loss: 0.0867 - val_accuracy: 0.9817
Epoch 966/1000
60000/60000 [==============================] - 2s 38us/step - loss: 3.1806e-04 - accuracy: 1.0000 - val_loss: 0.0868 - val_accuracy: 0.9816
Epoch 967/1000
60000/60000 [==============================] - 2s 38us/step - loss: 3.1757e-04 - accuracy: 1.0000 - val_loss: 0.0869 - val_accuracy: 0.9815
Epoch 968/1000
60000/60000 [==============================] - 2s 39us/step - loss: 3.1715e-04 - accuracy: 1.0000 - val_loss: 0.0868 - val_accuracy: 0.9816
Epoch 969/1000
60000/60000 [==============================] - 2s 38us/

In [4]:
# Randomly permute the class labels

y_train = np.random.permutation(y_train)
y_test = np.random.permutation(y_test)

In [22]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_17 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_18 (Dense)             (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [24]:
epochs = 1000

In [25]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 2s 40us/step - loss: 2.3118 - accuracy: 0.1024 - val_loss: 2.3093 - val_accuracy: 0.1024
Epoch 2/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.3052 - accuracy: 0.1068 - val_loss: 2.3085 - val_accuracy: 0.1005
Epoch 3/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.3021 - accuracy: 0.1104 - val_loss: 2.3082 - val_accuracy: 0.0979
Epoch 4/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.2999 - accuracy: 0.1141 - val_loss: 2.3076 - val_accuracy: 0.1007
Epoch 5/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.2980 - accuracy: 0.1159 - val_loss: 2.3074 - val_accuracy: 0.1014
Epoch 6/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.2963 - accuracy: 0.1194 - val_loss: 2.3076 - val_accuracy: 0.1042
Epoch 7/1000
60000/60000 [==============================

60000/60000 [==============================] - 2s 38us/step - loss: 2.0715 - accuracy: 0.2668 - val_loss: 2.4343 - val_accuracy: 0.0957
Epoch 110/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.0674 - accuracy: 0.2681 - val_loss: 2.4402 - val_accuracy: 0.0982
Epoch 111/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.0635 - accuracy: 0.2701 - val_loss: 2.4452 - val_accuracy: 0.0999
Epoch 112/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.0594 - accuracy: 0.2708 - val_loss: 2.4515 - val_accuracy: 0.0955
Epoch 113/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.0560 - accuracy: 0.2736 - val_loss: 2.4522 - val_accuracy: 0.0949
Epoch 114/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.0517 - accuracy: 0.2744 - val_loss: 2.4551 - val_accuracy: 0.0981
Epoch 115/1000
60000/60000 [==============================] - 2s 38us/step - loss: 2.0471 - accuracy: 0.2783 

60000/60000 [==============================] - 2s 40us/step - loss: 1.5618 - accuracy: 0.4670 - val_loss: 3.0226 - val_accuracy: 0.0990
Epoch 218/1000
60000/60000 [==============================] - 2s 38us/step - loss: 1.5526 - accuracy: 0.4713 - val_loss: 3.0335 - val_accuracy: 0.1014
Epoch 219/1000
60000/60000 [==============================] - 2s 37us/step - loss: 1.5481 - accuracy: 0.4731 - val_loss: 3.0119 - val_accuracy: 0.1057
Epoch 220/1000
60000/60000 [==============================] - 2s 38us/step - loss: 1.5433 - accuracy: 0.4744 - val_loss: 3.0433 - val_accuracy: 0.0983
Epoch 221/1000
60000/60000 [==============================] - 2s 38us/step - loss: 1.5384 - accuracy: 0.4781 - val_loss: 3.0732 - val_accuracy: 0.1015
Epoch 222/1000
60000/60000 [==============================] - 2s 37us/step - loss: 1.5358 - accuracy: 0.4769 - val_loss: 3.0604 - val_accuracy: 0.1029
Epoch 223/1000
60000/60000 [==============================] - 2s 38us/step - loss: 1.5277 - accuracy: 0.4812 

60000/60000 [==============================] - 2s 37us/step - loss: 1.0641 - accuracy: 0.6482 - val_loss: 3.9896 - val_accuracy: 0.0974
Epoch 326/1000
60000/60000 [==============================] - 2s 37us/step - loss: 1.0611 - accuracy: 0.6488 - val_loss: 4.0777 - val_accuracy: 0.0987
Epoch 327/1000
60000/60000 [==============================] - 2s 37us/step - loss: 1.0606 - accuracy: 0.6487 - val_loss: 4.1210 - val_accuracy: 0.1029
Epoch 328/1000
60000/60000 [==============================] - 2s 37us/step - loss: 1.0592 - accuracy: 0.6488 - val_loss: 4.0857 - val_accuracy: 0.1010
Epoch 329/1000
60000/60000 [==============================] - 2s 39us/step - loss: 1.0541 - accuracy: 0.6493 - val_loss: 4.0331 - val_accuracy: 0.1012
Epoch 330/1000
60000/60000 [==============================] - 2s 39us/step - loss: 1.0530 - accuracy: 0.6513 - val_loss: 4.1118 - val_accuracy: 0.0983
Epoch 331/1000
60000/60000 [==============================] - 2s 37us/step - loss: 1.0421 - accuracy: 0.6550 

60000/60000 [==============================] - 2s 37us/step - loss: 0.6802 - accuracy: 0.7824 - val_loss: 5.4316 - val_accuracy: 0.1011
Epoch 434/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.6821 - accuracy: 0.7805 - val_loss: 5.4375 - val_accuracy: 0.1007
Epoch 435/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.6726 - accuracy: 0.7873 - val_loss: 5.4628 - val_accuracy: 0.0988
Epoch 436/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.6741 - accuracy: 0.7851 - val_loss: 5.4449 - val_accuracy: 0.1001
Epoch 437/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.6672 - accuracy: 0.7874 - val_loss: 5.5216 - val_accuracy: 0.1009
Epoch 438/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.6645 - accuracy: 0.7891 - val_loss: 5.5262 - val_accuracy: 0.1001
Epoch 439/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.6543 - accuracy: 0.7942 

...

60000/60000 [==============================] - 2s 41us/step - loss: 0.1345 - accuracy: 0.9731 - val_loss: 10.0745 - val_accuracy: 0.0990
Epoch 704/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.1335 - accuracy: 0.9740 - val_loss: 10.1507 - val_accuracy: 0.0982
Epoch 705/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.1331 - accuracy: 0.9735 - val_loss: 10.1289 - val_accuracy: 0.0991
Epoch 706/1000
60000/60000 [==============================] - 2s 40us/step - loss: 0.1367 - accuracy: 0.9728 - val_loss: 10.1723 - val_accuracy: 0.0979
Epoch 707/1000
60000/60000 [==============================] - 2s 39us/step - loss: 0.1347 - accuracy: 0.9731 - val_loss: 10.1436 - val_accuracy: 0.0996
Epoch 708/1000
60000/60000 [==============================] - 2s 40us/step - loss: 0.1308 - accuracy: 0.9743 - val_loss: 10.1914 - val_accuracy: 0.0986
Epoch 709/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.1311 - accuracy: 0

Epoch 757/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.1020 - accuracy: 0.9811 - val_loss: 10.8117 - val_accuracy: 0.0998
Epoch 758/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.1019 - accuracy: 0.9810 - val_loss: 10.8413 - val_accuracy: 0.0989
Epoch 759/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.1008 - accuracy: 0.9816 - val_loss: 10.8663 - val_accuracy: 0.0982
Epoch 760/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.1020 - accuracy: 0.9820 - val_loss: 10.8715 - val_accuracy: 0.0995
Epoch 761/1000
60000/60000 [==============================] - 2s 39us/step - loss: 0.0995 - accuracy: 0.9823 - val_loss: 10.8957 - val_accuracy: 0.0984
Epoch 762/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0992 - accuracy: 0.9826 - val_loss: 10.9561 - val_accuracy: 0.0996
Epoch 763/1000
60000/60000 [==============================] - 2s 39us/step - loss: 0.098

Epoch 811/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0797 - accuracy: 0.9869 - val_loss: 11.4704 - val_accuracy: 0.0994
Epoch 812/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0796 - accuracy: 0.9869 - val_loss: 11.5089 - val_accuracy: 0.0975
Epoch 813/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0785 - accuracy: 0.9876 - val_loss: 11.5211 - val_accuracy: 0.0985
Epoch 814/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0775 - accuracy: 0.9873 - val_loss: 11.5441 - val_accuracy: 0.0988
Epoch 815/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0770 - accuracy: 0.9883 - val_loss: 11.5690 - val_accuracy: 0.0982
Epoch 816/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0768 - accuracy: 0.9875 - val_loss: 11.5637 - val_accuracy: 0.0994
Epoch 817/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.076

Epoch 918/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0474 - accuracy: 0.9951 - val_loss: 12.8479 - val_accuracy: 0.0985
Epoch 919/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0476 - accuracy: 0.9951 - val_loss: 12.8586 - val_accuracy: 0.0979
Epoch 920/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0469 - accuracy: 0.9956 - val_loss: 12.8778 - val_accuracy: 0.0988
Epoch 921/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0468 - accuracy: 0.9952 - val_loss: 12.8668 - val_accuracy: 0.0980
Epoch 922/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0464 - accuracy: 0.9956 - val_loss: 12.9076 - val_accuracy: 0.0976
Epoch 923/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0465 - accuracy: 0.9955 - val_loss: 12.9113 - val_accuracy: 0.0989
Epoch 924/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.046

Epoch 972/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0378 - accuracy: 0.9973 - val_loss: 13.4427 - val_accuracy: 0.0976
Epoch 973/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0377 - accuracy: 0.9973 - val_loss: 13.4629 - val_accuracy: 0.1001
Epoch 974/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0371 - accuracy: 0.9976 - val_loss: 13.4663 - val_accuracy: 0.0996
Epoch 975/1000
60000/60000 [==============================] - 2s 38us/step - loss: 0.0374 - accuracy: 0.9974 - val_loss: 13.4624 - val_accuracy: 0.0972
Epoch 976/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0368 - accuracy: 0.9975 - val_loss: 13.4788 - val_accuracy: 0.0981
Epoch 977/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.0367 - accuracy: 0.9976 - val_loss: 13.5042 - val_accuracy: 0.0979
Epoch 978/1000
60000/60000 [==============================] - 2s 37us/step - loss: 0.036

In [26]:
from keras.regularizers import l2

In [27]:
model1 = Sequential()
model1.add(Dense(256, activation='relu', input_shape=(784,), kernel_regularizer=l2(0.0001)))
model1.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0001)))
model1.add(Dense(num_classes, activation='softmax'))

model1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_20 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_21 (Dense)             (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [28]:
model1.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [29]:
epochs = 1000

In [30]:
history = model1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1000
60000/60000 [==============================] - 3s 48us/step - loss: 2.3765 - accuracy: 0.1014 - val_loss: 2.3723 - val_accuracy: 0.0983
Epoch 2/1000
60000/60000 [==============================] - 3s 44us/step - loss: 2.3684 - accuracy: 0.1088 - val_loss: 2.3711 - val_accuracy: 0.1012
Epoch 3/1000
60000/60000 [==============================] - 3s 45us/step - loss: 2.3652 - accuracy: 0.1128 - val_loss: 2.3704 - val_accuracy: 0.1023
Epoch 4/1000
60000/60000 [==============================] - 3s 46us/step - loss: 2.3631 - accuracy: 0.1146 - val_loss: 2.3705 - val_accuracy: 0.1012
Epoch 5/1000
60000/60000 [==============================] - 3s 42us/step - loss: 2.3614 - accuracy: 0.1161 - val_loss: 2.3698 - val_accuracy: 0.0981
Epoch 6/1000
60000/60000 [==============================] - 3s 42us/step - loss: 2.3597 - accuracy: 0.1189 - val_loss: 2.3699 - val_accuracy: 0.0997
Epoch 7/1000
60000/60000 [==============================

60000/60000 [==============================] - 3s 43us/step - loss: 2.1624 - accuracy: 0.2541 - val_loss: 2.4711 - val_accuracy: 0.0967
Epoch 110/1000
60000/60000 [==============================] - 3s 42us/step - loss: 2.1595 - accuracy: 0.2556 - val_loss: 2.4725 - val_accuracy: 0.0964
Epoch 111/1000
60000/60000 [==============================] - 3s 42us/step - loss: 2.1558 - accuracy: 0.2564 - val_loss: 2.4766 - val_accuracy: 0.0994
Epoch 112/1000
60000/60000 [==============================] - 3s 43us/step - loss: 2.1523 - accuracy: 0.2575 - val_loss: 2.4768 - val_accuracy: 0.0986
Epoch 113/1000
60000/60000 [==============================] - 3s 42us/step - loss: 2.1492 - accuracy: 0.2583 - val_loss: 2.4793 - val_accuracy: 0.0961
Epoch 114/1000
60000/60000 [==============================] - 3s 42us/step - loss: 2.1457 - accuracy: 0.2600 - val_loss: 2.4897 - val_accuracy: 0.0976
Epoch 115/1000
60000/60000 [==============================] - 3s 43us/step - loss: 2.1424 - accuracy: 0.2613 

60000/60000 [==============================] - 2s 42us/step - loss: 1.7264 - accuracy: 0.4414 - val_loss: 2.9997 - val_accuracy: 0.0952
Epoch 218/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.7227 - accuracy: 0.4414 - val_loss: 3.0466 - val_accuracy: 0.0947
Epoch 219/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.7183 - accuracy: 0.4430 - val_loss: 2.9993 - val_accuracy: 0.0967
Epoch 220/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.7129 - accuracy: 0.4460 - val_loss: 3.0453 - val_accuracy: 0.0998
Epoch 221/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.7068 - accuracy: 0.4475 - val_loss: 3.0283 - val_accuracy: 0.0965
Epoch 222/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.7071 - accuracy: 0.4463 - val_loss: 3.0368 - val_accuracy: 0.0966
Epoch 223/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.7000 - accuracy: 0.4530 

60000/60000 [==============================] - 3s 42us/step - loss: 1.3210 - accuracy: 0.6086 - val_loss: 3.8122 - val_accuracy: 0.0984
Epoch 326/1000
60000/60000 [==============================] - 3s 42us/step - loss: 1.3202 - accuracy: 0.6097 - val_loss: 3.8565 - val_accuracy: 0.0946
Epoch 327/1000
60000/60000 [==============================] - 3s 42us/step - loss: 1.3171 - accuracy: 0.6086 - val_loss: 3.8524 - val_accuracy: 0.0978
Epoch 328/1000
60000/60000 [==============================] - 3s 42us/step - loss: 1.3144 - accuracy: 0.6098 - val_loss: 3.8276 - val_accuracy: 0.0948
Epoch 329/1000
60000/60000 [==============================] - 3s 42us/step - loss: 1.3120 - accuracy: 0.6124 - val_loss: 3.9536 - val_accuracy: 0.0967
Epoch 330/1000
60000/60000 [==============================] - 3s 43us/step - loss: 1.3056 - accuracy: 0.6141 - val_loss: 3.9270 - val_accuracy: 0.0951
Epoch 331/1000
60000/60000 [==============================] - 3s 43us/step - loss: 1.3074 - accuracy: 0.6141 

60000/60000 [==============================] - 2s 41us/step - loss: 1.0189 - accuracy: 0.7374 - val_loss: 4.9837 - val_accuracy: 0.1005
Epoch 434/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.0276 - accuracy: 0.7317 - val_loss: 4.9288 - val_accuracy: 0.0994
Epoch 435/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.0221 - accuracy: 0.7355 - val_loss: 4.9377 - val_accuracy: 0.0986
Epoch 436/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.0134 - accuracy: 0.7408 - val_loss: 4.9230 - val_accuracy: 0.0975
Epoch 437/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.0168 - accuracy: 0.7402 - val_loss: 4.9272 - val_accuracy: 0.0966
Epoch 438/1000
60000/60000 [==============================] - 2s 41us/step - loss: 1.0202 - accuracy: 0.7374 - val_loss: 4.9858 - val_accuracy: 0.1016
Epoch 439/1000
60000/60000 [==============================] - 3s 42us/step - loss: 1.0097 - accuracy: 0.7416 

60000/60000 [==============================] - 2s 42us/step - loss: 0.8151 - accuracy: 0.8302 - val_loss: 6.0687 - val_accuracy: 0.0946
Epoch 542/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.8064 - accuracy: 0.8314 - val_loss: 6.1558 - val_accuracy: 0.0977
Epoch 543/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.8086 - accuracy: 0.8335 - val_loss: 6.0598 - val_accuracy: 0.0950
Epoch 544/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.8165 - accuracy: 0.8268 - val_loss: 6.0611 - val_accuracy: 0.0947
Epoch 545/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.8122 - accuracy: 0.8309 - val_loss: 6.0988 - val_accuracy: 0.0980
Epoch 546/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.7942 - accuracy: 0.8384 - val_loss: 6.0959 - val_accuracy: 0.0989
Epoch 547/1000
60000/60000 [==============================] - 3s 42us/step - loss: 0.7960 - accuracy: 0.8363 

60000/60000 [==============================] - 2s 41us/step - loss: 0.6645 - accuracy: 0.8982 - val_loss: 7.1300 - val_accuracy: 0.0967
Epoch 650/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.6591 - accuracy: 0.9018 - val_loss: 7.2026 - val_accuracy: 0.0980
Epoch 651/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.6544 - accuracy: 0.9044 - val_loss: 7.2580 - val_accuracy: 0.0965
Epoch 652/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.6666 - accuracy: 0.8979 - val_loss: 7.2320 - val_accuracy: 0.0974
Epoch 653/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.6698 - accuracy: 0.8971 - val_loss: 7.5222 - val_accuracy: 0.0978
Epoch 654/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.6807 - accuracy: 0.8902 - val_loss: 7.1824 - val_accuracy: 0.0986
Epoch 655/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.6475 - accuracy: 0.9071 

60000/60000 [==============================] - 2s 41us/step - loss: 0.5831 - accuracy: 0.9348 - val_loss: 8.0310 - val_accuracy: 0.0996
Epoch 758/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.5504 - accuracy: 0.9476 - val_loss: 8.0515 - val_accuracy: 0.0984
Epoch 759/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.5498 - accuracy: 0.9481 - val_loss: 8.0470 - val_accuracy: 0.0964
Epoch 760/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.5615 - accuracy: 0.9423 - val_loss: 8.0195 - val_accuracy: 0.0995
Epoch 761/1000
60000/60000 [==============================] - 3s 44us/step - loss: 0.5485 - accuracy: 0.9485 - val_loss: 8.1164 - val_accuracy: 0.0999
Epoch 762/1000
60000/60000 [==============================] - 3s 44us/step - loss: 0.5561 - accuracy: 0.9461 - val_loss: 8.0916 - val_accuracy: 0.0986
Epoch 763/1000
60000/60000 [==============================] - 3s 43us/step - loss: 0.5653 - accuracy: 0.9423 

60000/60000 [==============================] - 3s 43us/step - loss: 0.7096 - accuracy: 0.8856 - val_loss: 8.4587 - val_accuracy: 0.0964
Epoch 866/1000
60000/60000 [==============================] - 3s 42us/step - loss: 0.5298 - accuracy: 0.9534 - val_loss: 8.5130 - val_accuracy: 0.0971
Epoch 867/1000
60000/60000 [==============================] - 3s 43us/step - loss: 0.4896 - accuracy: 0.9690 - val_loss: 8.5129 - val_accuracy: 0.0992
Epoch 868/1000
60000/60000 [==============================] - 3s 44us/step - loss: 0.4862 - accuracy: 0.9702 - val_loss: 8.5400 - val_accuracy: 0.0958
Epoch 869/1000
60000/60000 [==============================] - 3s 43us/step - loss: 0.4828 - accuracy: 0.9711 - val_loss: 8.5546 - val_accuracy: 0.1008
Epoch 870/1000
60000/60000 [==============================] - 3s 43us/step - loss: 0.4768 - accuracy: 0.9730 - val_loss: 8.6687 - val_accuracy: 0.0992
Epoch 871/1000
60000/60000 [==============================] - 3s 43us/step - loss: 0.4810 - accuracy: 0.9715 

60000/60000 [==============================] - 2s 41us/step - loss: 0.4453 - accuracy: 0.9809 - val_loss: 8.7587 - val_accuracy: 0.0976
Epoch 974/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.4446 - accuracy: 0.9811 - val_loss: 8.8181 - val_accuracy: 0.0966
Epoch 975/1000
60000/60000 [==============================] - 2s 42us/step - loss: 0.4427 - accuracy: 0.9810 - val_loss: 8.7936 - val_accuracy: 0.1001
Epoch 976/1000
60000/60000 [==============================] - 2s 41us/step - loss: 0.4595 - accuracy: 0.9744 - val_loss: 8.7661 - val_accuracy: 0.0992
Epoch 977/1000
60000/60000 [==============================] - 2s 42us/step - loss: 0.5625 - accuracy: 0.9448 - val_loss: 9.3130 - val_accuracy: 0.0965
Epoch 978/1000
60000/60000 [==============================] - 2s 42us/step - loss: 0.9930 - accuracy: 0.7884 - val_loss: 8.6851 - val_accuracy: 0.0981
Epoch 979/1000
60000/60000 [==============================] - 3s 42us/step - loss: 0.7296 - accuracy: 0.8731 